In [101]:
import ujson
import re
import pandas as pd

from tqdm import tqdm_notebook
from glob import glob
from gensim.models.word2vec import Word2Vec
from itertools import product
from wordfreq import top_n_list
from IPython.display import Markdown, HTML, display
from tabulate import tabulate

In [117]:
pd.options.display.max_rows = 1000

In [18]:
def read_tweets(pattern):
    
    tweets = []
    for path in glob(pattern):
        with open(path) as fh:
            for line in tqdm_notebook(fh):
                
                tweet = ujson.loads(line)
                
                text = re.sub('(#|@|http)\S+', '', tweet['body'])
                tokens = re.findall('[a-z0-9]+', text.lower())
                
                tweets.append(tokens)
                
    return tweets

In [19]:
rural = read_tweets('../../data/geo-lt10k.json/part-0000*')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [15]:
urban = read_tweets('../../data/geo-gt1m.json/part-0000*')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [20]:
rural_model = Word2Vec(rural, workers=8)

In [22]:
urban_model = Word2Vec(urban, workers=8)

In [21]:
rural_model.most_similar('earth')

[('planet', 0.7829978466033936),
 ('dwell', 0.5269863605499268),
 ('world', 0.5174082517623901),
 ('universe', 0.5011920928955078),
 ('purpose', 0.4989180564880371),
 ('depending', 0.49261489510536194),
 ('mankind', 0.48292750120162964),
 ('christ', 0.47708719968795776),
 ('visible', 0.46374422311782837),
 ('species', 0.4624600410461426)]

In [23]:
urban_model.most_similar('earth')

[('planet', 0.8616721034049988),
 ('world', 0.5897253751754761),
 ('side', 0.5761807560920715),
 ('ocean', 0.574997067451477),
 ('purpose', 0.5739495158195496),
 ('soil', 0.5735812187194824),
 ('darkness', 0.5720939636230469),
 ('dwell', 0.5691649913787842),
 ('surface', 0.5686110854148865),
 ('path', 0.5609077215194702)]

In [52]:
def score_shift(m1, m2, token, n=10):
    
    m1_similar = [t for t, _ in m1.most_similar(token, topn=n)]
    m2_similar = [t for t, _ in m2.most_similar(token, topn=n)]
    
    score = 0
    for t1, t2 in product(m1_similar, m2_similar):
        score += m2.similarity(t1, t2)
        
    return score

In [53]:
score_shift(urban_model, rural_model, 'fuck')

49.193618851642924

In [115]:
data = []

for token in top_n_list('en', 1000):
    try:
        score = score_shift(urban_model, rural_model, token)
        data.append((token, score))
    except: pass

In [116]:
df = pd.DataFrame(data, columns=('token', 'score'))

In [118]:
df.sort_values('score').head(200)

,token,score
832,mark,13.427977
512,main,14.143367
471,march,15.087607
774,race,16.076421
781,w,17.017894
509,especially,19.902497
387,single,21.832514
803,boys,21.979826
102,here,22.061236
669,official,23.013374


In [119]:
def compare(token):
    
    u_similar = urban_model.most_similar(token)
    r_similar = rural_model.most_similar(token)
    
    data = []
    for (ut, _), (rt, _) in zip(u_similar, r_similar):
        data.append((ut, rt))
        
    return pd.DataFrame(data, columns=('urban', 'rural'))

In [125]:
for token in df.sort_values('score').head(100)['token']:
    display(Markdown(f'## {token}'))
    display(compare(token))

## mark

,urban,rural
0,goal,shania
1,second,calendars
2,first,rev
3,final,unshakeable
4,third,calenders
5,period,john
6,3rd,schaefer
7,fourth,gary
8,1st,legacy
9,last,lewis


## main

,urban,rural
0,biggest,coronation
1,first,street
2,sole,closing
3,only,located
4,current,current
5,prime,hub
6,weakest,wazers
7,whole,crossing
8,final,avalon
9,greatest,nj


## march

...

,urban,rural
0,lord,lord
1,goodness,jesus
2,od,christ
3,gosh,gods
4,gawd,faith
5,man,sins
6,heavens,salvation
7,gods,kindness
8,soul,patience
9,heart,forgiveness


## last

,urban,rural
0,first,first
1,past,next
2,ago,past
3,next,ago
4,every,earlier
5,1st,longest
6,ast,every
7,almost,previous
8,earlier,during
9,final,this


## against

,urban,rural
0,despite,versus
1,toward,defeat
2,towards,forces
3,facing,despite
4,domestic,battles
5,for,opponents
6,court,defend
7,defeat,defending
8,with,facing
9,forces,oppose
